In [31]:
import sympy as sp
from sympy import symbols, integrate, expand


# Codim ABD Quantity Derivation

This notebook derives the mass-matrix quantities for **codimensional** Affine Body Dynamics (ABD).

Recall that for a full 3D ABD body, the mass matrix $M \in \mathbb{R}^{12\times12}$ is:

$$M = \int_\Omega \rho\, J^T J\, dV$$

where $J$ is the $3\times12$ Jacobian with $x = J q$. Expanding $J^TJ$, we need three building blocks:

| Quantity | Definition |
|---|---|
| $m$ | $\int_\Omega \rho\, dV$ |
| $mx^\alpha$ | $\int_\Omega \rho\, x^\alpha\, dV$ |
| $mx^\alpha x^\beta$ | $\int_\Omega \rho\, x^\alpha x^\beta\, dV$ |

For codim objects, the **volume element** $dV$ is replaced by the appropriate lower-dimensional measure:

| Object | Mesh primitive | Jacobian factor | Density |
|---|---|---|---|
| 3D body | tetrahedron | $\det(e_1,e_2,e_3)$ | $\rho$ (volume) |
| 2D shell | triangle | $\|e_1\times e_2\|$ | $\rho_s$ (surface) |
| 1D rod | segment | $\|e_1\|$ | $\rho_l$ (linear) |

The 12 DOFs $q = [t_1,t_2,t_3,\, A_{11},A_{12},A_{13},\, A_{21},A_{22},A_{23},\, A_{31},A_{32},A_{33}]^T$ are the **same** for all cases — only the integration domain changes.


---
# Shell (Codim 2D)

A shell is a 2D surface embedded in 3D. We discretize it with a **triangle mesh** and assign
a **surface mass density** $\rho_s$ (mass per unit area).

For a triangle with vertices $p_0, p_1, p_2$, define:
$$e_1 = p_1 - p_0, \quad e_2 = p_2 - p_0$$

Any point on the triangle is parameterized as:
$$p = p_0 + e_1 u + e_2 v, \quad u,v \ge 0,\; u+v \le 1$$

The area element is:
$$dA = |e_1 \times e_2|\, du\, dv$$

where $|e_1 \times e_2|$ is a **constant** for each triangle (the Jacobian of the parametric map).

The per-triangle contributions are:
$$m_i = \rho_s\,|e_1\times e_2|_i\int_{\Omega_i} du\,dv$$
$$mx^\alpha_i = \rho_s\,|e_1\times e_2|_i\int_{\Omega_i} x^\alpha\, du\,dv$$
$$mx^\alpha x^\beta_i = \rho_s\,|e_1\times e_2|_i\int_{\Omega_i} x^\alpha x^\beta\, du\,dv$$

We integrate over the reference triangle $\{u,v \ge 0,\; u+v \le 1\}$.


In [32]:
u, v, t = symbols('u v t')

# Triangle vertices (for shell)
p_0a, p_0b = symbols('p_0^alpha p_0^beta')
p_1a, p_1b = symbols('p_1^alpha p_1^beta')
p_2a, p_2b = symbols('p_2^alpha p_2^beta')

# Edge vectors for triangle: e_k = p_k - p_0
e_1a, e_1b = symbols('e_1^alpha e_1^beta')
e_2a, e_2b = symbols('e_2^alpha e_2^beta')

# Expressions for edge vectors in terms of vertices
expr_e_1a, expr_e_1b = p_1a - p_0a, p_1b - p_0b
expr_e_2a, expr_e_2b = p_2a - p_0a, p_2b - p_0b

def integrate_S(f):
    """Integrate f(u,v) over the reference triangle {u,v >= 0, u+v <= 1}"""
    fv = integrate(f, (u, 0, 1-v))
    f  = integrate(fv, (v, 0, 1))
    return f


### Shell: Area (mass coefficient)

$$\int_{\Omega_i} du\,dv = ?$$


In [33]:
I_S = integrate_S(1)
I_S


1/2

$$\int_{\Omega_i} du\,dv = \frac{1}{2}$$

So the area of triangle $i$ is:
$$A_i = \frac{1}{2}|e_1 \times e_2|_i$$

and the mass:
$$m_i = \frac{\rho_s}{2}\,|e_1\times e_2|_i$$

---
### Shell: First Moment $\int x^\alpha\, du\,dv$


In [34]:
# Edge form
integrate_S(p_0a + e_1a*u + e_2a*v)


e_1^alpha/6 + e_2^alpha/6 + p_0^alpha/2

In [35]:
# Vertex form (substitute e_k = p_k - p_0)
integrate_S(p_0a + expr_e_1a*u + expr_e_2a*v)


p_0^alpha/6 + p_1^alpha/6 + p_2^alpha/6

$$\int_{\Omega_i} x^\alpha\, du\,dv = \frac{p_0^\alpha + p_1^\alpha + p_2^\alpha}{6}$$

So the per-triangle first moment:
$$mx^\alpha_i = \rho_s\, |e_1\times e_2|_i \cdot \frac{p_0^\alpha + p_1^\alpha + p_2^\alpha}{6} = m_i \cdot \underbrace{\frac{p_0^\alpha + p_1^\alpha + p_2^\alpha}{3}}_{\text{centroid}^\alpha}$$

This makes sense: the first moment equals mass times the centroid of the triangle.

---
### Shell: Second Moment $\int x^\alpha x^\beta\, du\,dv$


In [36]:
# Edge form
p_a = p_0a + e_1a*u + e_2a*v
p_b = p_0b + e_1b*u + e_2b*v
P = expand(p_a * p_b)

integrate_S(P)


e_1^alpha*e_1^beta/12 + e_1^alpha*e_2^beta/24 + e_1^alpha*p_0^beta/6 + e_1^beta*e_2^alpha/24 + e_1^beta*p_0^alpha/6 + e_2^alpha*e_2^beta/12 + e_2^alpha*p_0^beta/6 + e_2^beta*p_0^alpha/6 + p_0^alpha*p_0^beta/2

In [37]:
# Vertex form
p_a = p_0a + expr_e_1a*u + expr_e_2a*v
p_b = p_0b + expr_e_1b*u + expr_e_2b*v
P = expand(p_a * p_b)

integrate_S(P)


p_0^alpha*p_0^beta/12 + p_0^alpha*p_1^beta/24 + p_0^alpha*p_2^beta/24 + p_0^beta*p_1^alpha/24 + p_0^beta*p_2^alpha/24 + p_1^alpha*p_1^beta/12 + p_1^alpha*p_2^beta/24 + p_1^beta*p_2^alpha/24 + p_2^alpha*p_2^beta/12

The second moment in vertex form reads off directly from sympy output:

$$
\int_{\Omega_i} x^\alpha x^\beta\, du\,dv
= \frac{2(p_0^\alpha p_0^\beta + p_1^\alpha p_1^\beta + p_2^\alpha p_2^\beta)
   + (p_0^\alpha p_1^\beta + p_1^\alpha p_0^\beta)
   + (p_0^\alpha p_2^\beta + p_2^\alpha p_0^\beta)
   + (p_1^\alpha p_2^\beta + p_2^\alpha p_1^\beta)}{24}
$$

### Shell: Summary

For each triangle $i$ with vertices $p_0, p_1, p_2$ and edge vectors $e_k = p_k - p_0$:

$$m_i = \frac{\rho_s}{2} |e_1 \times e_2|_i$$

$$mx^\alpha_i = \rho_s |e_1 \times e_2|_i \cdot \frac{p_0^\alpha + p_1^\alpha + p_2^\alpha}{6}$$

$$mx^\alpha x^\beta_i = \rho_s |e_1 \times e_2|_i \cdot \frac{2\sum_j p_j^\alpha p_j^\beta + \sum_{j \neq k} p_j^\alpha p_k^\beta}{24}$$

The global quantities are sums over all triangles.


---
# Rod (Codim 1D)

A rod is a 1D curve embedded in 3D. We discretize it with a **segment mesh** (polyline) and assign
a **linear mass density** $\rho_l$ (mass per unit length).

For a segment with endpoints $p_0, p_1$, define:
$$e_1 = p_1 - p_0$$

Any point on the segment is parameterized as:
$$p = p_0 + e_1 t, \quad t \in [0, 1]$$

The length element is:
$$dL = |e_1|\, dt$$

where $|e_1|$ is a **constant** for each segment (the Jacobian of the parametric map).

The per-segment contributions are:
$$m_i = \rho_l\,|e_1|_i \int_{0}^{1} dt$$
$$mx^\alpha_i = \rho_l\,|e_1|_i \int_{0}^{1} x^\alpha\, dt$$
$$mx^\alpha x^\beta_i = \rho_l\,|e_1|_i \int_{0}^{1} x^\alpha x^\beta\, dt$$


In [ ]:
# Segment endpoints
p_0a_r, p_0b_r = symbols('p_0^alpha p_0^beta')
p_1a_r, p_1b_r = symbols('p_1^alpha p_1^beta')

# Edge vector: e_1 = p_1 - p_0
e_1a_r, e_1b_r = symbols('e_1^alpha e_1^beta')

# Expression for edge vector in terms of vertices
expr_e_1a_r = p_1a_r - p_0a_r
expr_e_1b_r = p_1b_r - p_0b_r

def integrate_L(f):
    """Integrate f(t) over the reference interval [0, 1]"""
    return integrate(f, (t, 0, 1))


### Rod: Length (mass coefficient)

$$\int_{0}^{1} dt = ?$$


In [39]:
I_L = integrate_L(1)
I_L


1

$$\int_{0}^{1} dt = 1$$

So the length of segment $i$ is $L_i = |e_1|_i$, and the mass:
$$m_i = \rho_l\, |e_1|_i$$

---
### Rod: First Moment $\int_0^1 x^\alpha\, dt$


In [40]:
# Edge form
integrate_L(p_0a_r + e_1a_r*t)


e_1^alpha/2 + p_0^alpha

In [41]:
# Vertex form
integrate_L(p_0a_r + expr_e_1a_r*t)


p_0^alpha/2 + p_1^alpha/2

$$\int_0^1 x^\alpha\, dt = \frac{p_0^\alpha + p_1^\alpha}{2}$$

So the per-segment first moment:
$$mx^\alpha_i = \rho_l\,|e_1|_i\cdot\frac{p_0^\alpha + p_1^\alpha}{2} = m_i\cdot\underbrace{\frac{p_0^\alpha + p_1^\alpha}{2}}_{\text{midpoint}^\alpha}$$

The first moment equals mass times the midpoint of the segment.

---
### Rod: Second Moment $\int_0^1 x^\alpha x^\beta\, dt$


In [42]:
# Edge form
p_a_r = p_0a_r + e_1a_r*t
p_b_r = p_0b_r + e_1b_r*t
P_r = expand(p_a_r * p_b_r)

integrate_L(P_r)


e_1^alpha*e_1^beta/3 + e_1^alpha*p_0^beta/2 + e_1^beta*p_0^alpha/2 + p_0^alpha*p_0^beta

In [43]:
# Vertex form
p_a_r = p_0a_r + expr_e_1a_r*t
p_b_r = p_0b_r + expr_e_1b_r*t
P_r = expand(p_a_r * p_b_r)

integrate_L(P_r)


p_0^alpha*p_0^beta/3 + p_0^alpha*p_1^beta/6 + p_0^beta*p_1^alpha/6 + p_1^alpha*p_1^beta/3

$$
\int_0^1 x^\alpha x^\beta\, dt
= \frac{2\,p_0^\alpha p_0^\beta + p_0^\alpha p_1^\beta + p_1^\alpha p_0^\beta + 2\,p_1^\alpha p_1^\beta}{6}
$$

Special case $\alpha = \beta$:
$$\int_0^1 (x^\alpha)^2\, dt = \frac{(p_0^\alpha)^2 + p_0^\alpha p_1^\alpha + (p_1^\alpha)^2}{3}$$

### Rod: Summary

For each segment $i$ with endpoints $p_0, p_1$ and edge vector $e_1 = p_1 - p_0$:

$$m_i = \rho_l\,|e_1|_i$$

$$mx^\alpha_i = \rho_l\,|e_1|_i\cdot\frac{p_0^\alpha + p_1^\alpha}{2}$$

$$mx^\alpha x^\beta_i = \rho_l\,|e_1|_i\cdot\frac{2\,p_0^\alpha p_0^\beta + p_0^\alpha p_1^\beta + p_1^\alpha p_0^\beta + 2\,p_1^\alpha p_1^\beta}{6}$$

The global quantities are sums over all segments.


---
# Comparison: 3D Body vs Shell vs Rod

| | **3D Body** (tetrahedra) | **2D Shell** (triangles) | **1D Rod** (segments) |
|---|---|---|---|
| Jacobian $J_i$ | $\det(e_1,e_2,e_3)$ | $\|e_1\times e_2\|$ | $\|e_1\|$ |
| $m_i \;/\; (\rho \cdot J_i)$ | $\tfrac{1}{6}$ | $\tfrac{1}{2}$ | $1$ |
| $mx^\alpha_i \;/\; (\rho \cdot J_i)$ | $\tfrac{\sum_{j=0}^3 p_j^\alpha}{24}$ | $\tfrac{\sum_{j=0}^2 p_j^\alpha}{6}$ | $\tfrac{p_0^\alpha + p_1^\alpha}{2}$ |
| Diagonal $p_j^\alpha p_j^\beta$ weight | $\tfrac{2}{120} = \tfrac{1}{60}$ | $\tfrac{2}{24} = \tfrac{1}{12}$ | $\tfrac{2}{6} = \tfrac{1}{3}$ |
| Off-diagonal $p_j^\alpha p_k^\beta$ weight ($j\neq k$) | $\tfrac{1}{120}$ | $\tfrac{1}{24}$ | $\tfrac{1}{6}$ |

Note: the diagonal-to-off-diagonal ratio is always $2:1$, reflecting the pattern
$$\frac{2\sum_j p_j^\alpha p_j^\beta + \sum_{j\neq k} p_j^\alpha p_k^\beta}{\text{denominator}}$$
which simply counts each pair twice for the diagonal and once for the off-diagonal terms.

The ratio of diagonal to off-diagonal coefficient is always $2:1$ across all cases, matching
the multinomial structure of the integral $\int (\sum_j p_j \phi_j)^2 d\Omega$ where $\phi_j$
are the barycentric basis functions.


---
# Invertibility of the 12x12 ABD Mass Matrix

## The problem

The ABD mass matrix $M \in \mathbb{R}^{12\times 12}$ is assembled from the three dyadic quantities $(m,\; m\bar{x},\; m\bar{x}\bar{x}^T)$
with the block structure (same block repeated for each spatial dimension $i=0,1,2$):

$$
B_i = \begin{bmatrix} m & m\bar{x}^T \\ m\bar{x} & m\bar{x}\bar{x}^T \end{bmatrix}_{4\times 4}
$$

For $M$ to be invertible, $m\bar{x}\bar{x}^T$ must be **full-rank** (3×3).

Consider a flat triangle in the $z=0$ plane. All rest positions have $\bar{x}_3 = 0$, so:

$$m\bar{x}\bar{x}^T = \begin{bmatrix} m_{xx} & m_{xy} & 0 \\ m_{yx} & m_{yy} & 0 \\ 0 & 0 & 0 \end{bmatrix}$$

The $z$-row/column is zero $\Rightarrow$ **rank 2**, not 3 $\Rightarrow$ $M$ is **singular**.

Physically: the DOF $\mathbf{a}_3$ (third row of affine matrix $A$) multiplies $\bar{x}_3 = 0$ for every point on the surface, so it has zero kinetic energy.

## The fix: integrate over the full 3D slab/cylinder

A shell is not truly 2D — it has physical **thickness** $2r$. A rod has a **cross-section** of radius $r$.
We must integrate $\rho J^T J$ over the actual 3D volume, not just the codim manifold.

**Convention**: `thickness` stores radius $r$. Shell slab extends $z \in [-r, r]$ in the normal direction; rod cylinder has radius $r$.


## Shell: Slab correction for $m\bar{x}\bar{x}^T$

For a triangle with unit normal $\hat{n}$, the slab extends from $-r$ to $+r$ in the normal direction.
A point in the slab is $\bar{x} = \bar{x}_{\text{surface}} + s\,\hat{n}$ where $s \in [-r, r]$.

The slab integral of $\bar{x}^\alpha \bar{x}^\beta$ splits into:

$$
\int_{\text{slab}} \rho\, \bar{x}^\alpha \bar{x}^\beta\, dV
= \underbrace{\rho \cdot 2r \int_{\text{triangle}} \bar{x}_s^\alpha \bar{x}_s^\beta\, dA}_{\text{in-plane (derived above)}}
+ \underbrace{\rho \cdot A_i \cdot \frac{2r^3}{3}\; (\hat{n} \otimes \hat{n})^{\alpha\beta}}_{\text{normal correction}}
$$

The cross terms $\bar{x}_s^\alpha \cdot s\,\hat{n}^\beta$ vanish because $\int_{-r}^{r} s\, ds = 0$ (symmetric slab).

The normal correction comes from:
$$\int_{-r}^{r} s^2\, ds = \frac{2r^3}{3}$$

Let's verify with sympy:


In [44]:
r, s = symbols('r s', positive=True)

# The symmetric slab integral that contributes to m_x_bar:
# int_{-r}^{r} s ds = 0 (no shift in centroid)
slab_first_moment = integrate(s, (s, -r, r))
print("Slab first moment (cross term):", slab_first_moment)

# The slab integral that contributes to m_x_bar_x_bar:
# int_{-r}^{r} s^2 ds = 2r^3/3 (normal-direction inertia)
slab_second_moment = integrate(s**2, (s, -r, r))
print("Slab second moment (normal correction):", slab_second_moment)


Slab first moment (cross term): 0
Slab second moment (normal correction): 2*r**3/3


### Shell: Corrected formulas (with thickness $r$)

Using volume density $\rho$ (kg/m$^3$) and effective surface density $\rho_s = \rho \cdot 2r$:

$$\boxed{m_i = \rho \cdot A_i \cdot 2r}$$

$$\boxed{m\bar{x}^\alpha_i = \rho \cdot |e_1 \times e_2|_i \cdot 2r \cdot \frac{p_0^\alpha + p_1^\alpha + p_2^\alpha}{6}}$$

$$\boxed{m\bar{x}^\alpha \bar{x}^\beta_i = \rho \cdot |e_1 \times e_2|_i \cdot 2r \cdot \frac{2\sum_j p_j^\alpha p_j^\beta + \sum_{j\neq k} p_j^\alpha p_k^\beta}{24}
\;+\; \rho \cdot A_i \cdot \frac{2r^3}{3}\; \hat{n}_i^\alpha \hat{n}_i^\beta}$$

where $\hat{n}_i = \frac{e_1 \times e_2}{|e_1 \times e_2|}$ is the unit normal of triangle $i$.

The first two terms are the in-plane contribution (= manifold integral $\times\, 2r$).
The last term is the **normal correction** from slab thickness — it fills the null-space direction.


## Rod: Cross-section correction for $m\bar{x}\bar{x}^T$

For a segment with unit tangent $\hat{t} = e_1/|e_1|$, choose two orthonormal vectors $\hat{e}_1, \hat{e}_2$ spanning the cross-section plane ($\hat{e}_1 \times \hat{e}_2 = \hat{t}$).

A point in the cylinder is $\bar{x} = \bar{x}_{\text{axis}}(t) + s_1\,\hat{e}_1 + s_2\,\hat{e}_2$ where $s_1^2 + s_2^2 \le r^2$.

The cross-section integral of the second moment splits similarly:

$$
\int_{\text{cylinder}} \rho\, \bar{x}^\alpha \bar{x}^\beta\, dV
= \underbrace{\rho \cdot \pi r^2 \cdot |e_1| \cdot (\text{1D formula})}_{\text{along-axis (derived above)}}
+ \underbrace{\rho \cdot L_i \cdot \frac{\pi r^4}{4}\; (I - \hat{t} \otimes \hat{t})^{\alpha\beta}}_{\text{cross-section correction}}
$$

The cross-section correction arises from the second moment of area of a disk:

$$\iint_{\text{disk}} s_1^2\, dA = \iint_{\text{disk}} s_2^2\, dA = \frac{\pi r^4}{4}$$

projected onto the plane perpendicular to $\hat{t}$ via $(I - \hat{t}\otimes\hat{t})$.

Let's verify the disk integral with sympy:


In [45]:
from sympy import pi as sym_pi, sqrt

s1, s2 = symbols('s_1 s_2')

# Second moment of area of a disk: int int_{s1^2+s2^2 <= r^2} s1^2 dA
# Use polar: s1 = rho*cos(theta), s2 = rho*sin(theta), dA = rho d_rho d_theta
rho_polar, theta = symbols('rho theta', positive=True)

I_disk_s1sq = integrate(
    integrate(rho_polar * (rho_polar * sp.cos(theta))**2,
              (rho_polar, 0, r)),
    (theta, 0, 2*sym_pi)
)
print("int int_{disk} s1^2 dA =", I_disk_s1sq)

# Cross term: int int_{disk} s1*s2 dA = 0 by symmetry
I_disk_s1s2 = integrate(
    integrate(rho_polar * (rho_polar * sp.cos(theta)) * (rho_polar * sp.sin(theta)),
              (rho_polar, 0, r)),
    (theta, 0, 2*sym_pi)
)
print("int int_{disk} s1*s2 dA =", I_disk_s1s2)


int int_{disk} s1^2 dA = pi*r**4/4
int int_{disk} s1*s2 dA = 0


### Rod: Corrected formulas (with thickness $r$)

Using volume density $\rho$ (kg/m$^3$), cross-section area $S = \pi r^2$, and second moment of area $I_{cs} = \pi r^4/4$:

$$\boxed{m_i = \rho \cdot L_i \cdot \pi r^2}$$

$$\boxed{m\bar{x}^\alpha_i = \rho \cdot |e_1|_i \cdot \pi r^2 \cdot \frac{p_0^\alpha + p_1^\alpha}{2}}$$

$$\boxed{m\bar{x}^\alpha \bar{x}^\beta_i = \rho \cdot |e_1|_i \cdot \pi r^2 \cdot \frac{2p_0^\alpha p_0^\beta + p_0^\alpha p_1^\beta + p_1^\alpha p_0^\beta + 2p_1^\alpha p_1^\beta}{6}
\;+\; \rho \cdot L_i \cdot \frac{\pi r^4}{4} \; (I - \hat{t}_i \otimes \hat{t}_i)^{\alpha\beta}}$$

where $\hat{t}_i = e_1/|e_1|$ is the unit tangent of segment $i$.

The first term is the along-axis contribution (= 1D integral $\times\, \pi r^2$).
The second term is the **cross-section correction** — it fills the two null-space directions perpendicular to the rod.


## Numerical verification: single triangle

Triangle $p_0=(0,0,0),\; p_1=(1,0,0),\; p_2=(0,1,0)$, with $\rho=1000$, $r=0.1$.

We verify that $m\bar{x}\bar{x}^T$ is full-rank and the resulting 12x12 mass matrix is positive definite.


In [46]:
import numpy as np

# Triangle vertices
p0 = np.array([0.0, 0.0, 0.0])
p1 = np.array([1.0, 0.0, 0.0])
p2 = np.array([0.0, 1.0, 0.0])

rho_val = 1000.0
r_val   = 0.1
twor    = 2.0 * r_val

e1 = p1 - p0
e2 = p2 - p0
cross = np.cross(e1, e2)
D = np.linalg.norm(cross)         # |e1 x e2| = 1
area = 0.5 * D                     # 0.5
n_hat = cross / D                   # (0, 0, 1)

# --- Mass ---
m_val = rho_val * area * twor
print(f"m = {m_val}")

# --- First moment ---
mx = rho_val * D * twor * (p0 + p1 + p2) / 6.0
print(f"m_x_bar = {mx}")

# --- Second moment (in-plane) ---
ps = [p0, p1, p2]
mxx_plane = np.zeros((3, 3))
for a in range(3):
    for b in range(3):
        val = 0.0
        for j in range(3):
            val += 2.0 * ps[j][a] * ps[j][b]  # diagonal
        for j in range(3):
            for k in range(3):
                if j != k:
                    val += ps[j][a] * ps[k][b]  # off-diagonal
        mxx_plane[a, b] = rho_val * D * twor * val / 24.0

# --- Normal correction ---
mxx_normal = rho_val * area * (2.0 * r_val**3 / 3.0) * np.outer(n_hat, n_hat)

mxx = mxx_plane + mxx_normal

print(f"\nm_x_bar_x_bar (in-plane only):\n{mxx_plane}")
print(f"\nm_x_bar_x_bar (normal correction):\n{mxx_normal}")
print(f"\nm_x_bar_x_bar (total):\n{mxx}")
print(f"\nrank(m_x_bar_x_bar, in-plane only) = {np.linalg.matrix_rank(mxx_plane)}")
print(f"rank(m_x_bar_x_bar, total)         = {np.linalg.matrix_rank(mxx)}")


m = 100.0
m_x_bar = [33.33333333 33.33333333  0.        ]

m_x_bar_x_bar (in-plane only):
[[16.66666667  8.33333333  0.        ]
 [ 8.33333333 16.66666667  0.        ]
 [ 0.          0.          0.        ]]

m_x_bar_x_bar (normal correction):
[[0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.33333333]]

m_x_bar_x_bar (total):
[[16.66666667  8.33333333  0.        ]
 [ 8.33333333 16.66666667  0.        ]
 [ 0.          0.          0.33333333]]

rank(m_x_bar_x_bar, in-plane only) = 2
rank(m_x_bar_x_bar, total)         = 3


In [47]:
# Build the 12x12 ABD mass matrix (same structure as ABDJacobiDyadicMass::add_to)
M = np.zeros((12, 12))
for i in range(3):
    M[i, i] += m_val
    M[i, 3+3*i:6+3*i] += mx
    M[3+3*i:6+3*i, i] += mx
    M[3+3*i:6+3*i, 3+3*i:6+3*i] += mxx

# Check positive definiteness via eigenvalues
eigvals = np.linalg.eigvalsh(M)
print("Eigenvalues of M (12x12):")
print(np.sort(eigvals))
print(f"\nAll eigenvalues > 0? {np.all(eigvals > 0)}")
print(f"Condition number: {eigvals.max() / eigvals.min():.2f}")


Eigenvalues of M (12x12):
[  0.33333333   0.33333333   0.33333333   2.26319877   2.26319877
   2.26319877   8.33333333   8.33333333   8.33333333 122.73680123
 122.73680123 122.73680123]

All eigenvalues > 0? True
Condition number: 368.21


## Numerical verification: single segment

Segment $p_0=(0,0,0)$, $p_1=(1,0,0)$, with $\rho=1000$, $r=0.1$.


In [48]:
# Segment vertices
p0_r = np.array([0.0, 0.0, 0.0])
p1_r = np.array([1.0, 0.0, 0.0])

rho_val = 1000.0
r_val   = 0.1
pi_val  = np.pi
S_val   = pi_val * r_val**2  # cross-section area

e1_r  = p1_r - p0_r
L_val = np.linalg.norm(e1_r)       # 1.0
t_hat = e1_r / L_val               # (1, 0, 0)

# --- Mass ---
m_rod = rho_val * L_val * S_val
print(f"m = {m_rod:.6f}")

# --- First moment ---
mx_rod = rho_val * L_val * S_val * (p0_r + p1_r) / 2.0
print(f"m_x_bar = {mx_rod}")

# --- Second moment (along-axis) ---
ps_r = [p0_r, p1_r]
mxx_axis = np.zeros((3, 3))
for a in range(3):
    for b in range(3):
        val = (2*p0_r[a]*p0_r[b] + p0_r[a]*p1_r[b]
               + p1_r[a]*p0_r[b] + 2*p1_r[a]*p1_r[b]) / 6.0
        mxx_axis[a, b] = rho_val * L_val * S_val * val

# --- Cross-section correction ---
Ics = pi_val * r_val**4 / 4.0
P_perp = np.eye(3) - np.outer(t_hat, t_hat)
mxx_cs = rho_val * L_val * Ics * P_perp

mxx_rod = mxx_axis + mxx_cs

print(f"\nm_x_bar_x_bar (along-axis only):\n{mxx_axis}")
print(f"\nm_x_bar_x_bar (cross-section correction):\n{mxx_cs}")
print(f"\nm_x_bar_x_bar (total):\n{mxx_rod}")
print(f"\nrank(m_x_bar_x_bar, axis only) = {np.linalg.matrix_rank(mxx_axis)}")
print(f"rank(m_x_bar_x_bar, total)     = {np.linalg.matrix_rank(mxx_rod)}")

# Build 12x12
M_rod = np.zeros((12, 12))
for i in range(3):
    M_rod[i, i] += m_rod
    M_rod[i, 3+3*i:6+3*i] += mx_rod
    M_rod[3+3*i:6+3*i, i] += mx_rod
    M_rod[3+3*i:6+3*i, 3+3*i:6+3*i] += mxx_rod

eigvals_rod = np.linalg.eigvalsh(M_rod)
print("\nEigenvalues of M (12x12):")
print(np.sort(eigvals_rod))
print(f"\nAll eigenvalues > 0? {np.all(eigvals_rod > 0)}")
print(f"Condition number: {eigvals_rod.max() / eigvals_rod.min():.2f}")


m = 31.415927
m_x_bar = [15.70796327  0.          0.        ]

m_x_bar_x_bar (along-axis only):
[[10.47197551  0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]

m_x_bar_x_bar (cross-section correction):
[[0.         0.         0.        ]
 [0.         0.07853982 0.        ]
 [0.         0.         0.07853982]]

m_x_bar_x_bar (total):
[[10.47197551  0.          0.        ]
 [ 0.          0.07853982  0.        ]
 [ 0.          0.          0.07853982]]

rank(m_x_bar_x_bar, axis only) = 1
rank(m_x_bar_x_bar, total)     = 3

Eigenvalues of M (12x12):
[ 0.07853982  0.07853982  0.07853982  0.07853982  0.07853982  0.07853982
  2.06532869  2.06532869  2.06532869 39.82257336 39.82257336 39.82257336]

All eigenvalues > 0? True
Condition number: 507.04


## Summary: Why the corrections are mandatory

| | Without correction | With correction |
|---|---|---|
| **Shell** $m\bar{x}\bar{x}^T$ | Rank 2 (zero in normal dir) | **Rank 3** (filled by $\rho A \frac{2r^3}{3} \hat{n}\otimes\hat{n}$) |
| **Rod** $m\bar{x}\bar{x}^T$ | Rank 1 (zero in cross-section) | **Rank 3** (filled by $\rho L \frac{\pi r^4}{4} (I - \hat{t}\otimes\hat{t})$) |
| 12x12 mass matrix $M$ | **Singular** | **Positive definite** |

The corrections are proportional to $r^3$ (shell) and $r^4$ (rod) vs the main terms proportional to $r$ and $r^2$.
They are small for thin bodies but **structurally essential** for the mass matrix to be invertible.

Physically, they represent the rotational inertia of the slab/cylinder about axes within the codim manifold.


---
# Body Force for Codim ABD

The ABD generalized body force is:

$$G = \int_\Omega J^T f_{\text{density}} \, dV$$

where $f_{\text{density}} = \rho\, g_{\text{accel}}$ is the body force per unit volume (N/m³) and $J$ is the $3\times12$ Jacobian.

From the J structure, the integral expands to:

$$G[0:3] = \int_\Omega f_{\text{density}} \, dV$$

$$G[3:6] = f_{\text{density},x} \int_\Omega \bar{x} \, dV, \qquad G[6:9] = f_{\text{density},y} \int_\Omega \bar{x} \, dV, \qquad G[9:12] = f_{\text{density},z} \int_\Omega \bar{x} \, dV$$

The key integrals are:

| Quantity | Symbol |
|---|---|
| $\int_\Omega dV$ | $V_{\text{eff}}$ (effective element volume) |
| $\int_\Omega \bar{x}\, dV$ | $V_{\text{eff}} \cdot c$ (effective volume $\times$ centroid) |

So the body force simplifies to:

$$G[0:3] = f_{\text{density}} \cdot V_{\text{eff}}$$
$$G[3:12] = \begin{bmatrix} f_x \\ f_y \\ f_z \end{bmatrix} \otimes (V_{\text{eff}} \cdot c)$$

This mirrors the existing 3D tet formula where $Q_s = V \cdot c$ (tet centroid $\times$ tet volume).

The codim cases add the cross-section/slab integration:

- **Shell**: $\int_{\text{slab}} \bar{x}^\alpha \, dV = \underbrace{2r \int_{\text{triangle}} x_s^\alpha \, dA}_{\text{surface}} + \underbrace{\hat{n}^\alpha \int_{-r}^{r} s\, ds}_{=0} = 2r \cdot |e_1 \times e_2| \cdot \frac{p_0^\alpha+p_1^\alpha+p_2^\alpha}{6}$
  
  So $\int_{\text{slab}} \bar{x} \, dV = V_{\text{slab}} \cdot c_{\triangle}$ where $c_\triangle = (p_0+p_1+p_2)/3$.

- **Rod**: $\int_{\text{cylinder}} \bar{x}^\alpha \, dV = \pi r^2 \cdot |e_1| \int_0^1 x_{\text{axis}}^\alpha \, dt + \int_{\text{disk}} s_1 \, dA \cdot \hat{e}_1^\alpha + \int_{\text{disk}} s_2 \, dA \cdot \hat{e}_2^\alpha$

  The disk integrals $\int_{\text{disk}} s_k \, dA = 0$ (centroid of disk is at center), so $\int_{\text{cylinder}} \bar{x} \, dV = V_{\text{cyl}} \cdot m_{\text{seg}}$ where $m_{\text{seg}} = (p_0+p_1)/2$.

Let's verify both with sympy:


In [ ]:
# --- Shell slab first moment ---
# int_{-r}^{r} s ds = 0  (cross term vanishes, no centroid shift)
r, s = symbols('r s', positive=True)
print("Slab cross term int s ds:", integrate(s, (s, -r, r)))

# In-plane integral: int_{triangle} x^a du dv (from earlier)
# = |e1xe2| * (p0^a + p1^a + p2^a) / 6
# Scaled by 2r:
# => int_slab x^a dV = 2r * |e1xe2| * (p0+p1+p2)/6 = V_slab * centroid

p_0a, p_1a, p_2a = symbols('p_0^alpha p_1^alpha p_2^alpha')
D_s = symbols('D_s', positive=True)  # |e1 x e2|

# V_slab = area * 2r = (D_s/2) * 2r = D_s * r
V_slab = D_s * r

# int_slab x^a dV = 2r * |e1xe2| * (surface integral result)
# From cell 8: integrate_S(x^a) = (p0+p1+p2)/6
surface_integral = (p_0a + p_1a + p_2a) / 6
first_moment_slab = 2 * r * D_s * surface_integral
centroid = (p_0a + p_1a + p_2a) / 3
print("int_slab x^a dV =", sp.simplify(first_moment_slab))
print("V_slab * centroid =", sp.simplify(V_slab * centroid))
print("Equal:", sp.simplify(first_moment_slab - V_slab * centroid) == 0)


In [ ]:
# --- Rod cylinder first moment ---
# Disk cross-section integrals (centroid of disk is at center)
from sympy import pi as sym_pi, cos, sin

rho_p, theta = symbols('rho theta', positive=True)

# int_disk s1 dA = 0  (s1 = rho*cos(theta))
I_disk_s1 = integrate(integrate(rho_p * rho_p * cos(theta), (rho_p, 0, r)), (theta, 0, 2*sym_pi))
print("int_disk s1 dA =", I_disk_s1)

# int_disk s2 dA = 0  (s2 = rho*sin(theta))
I_disk_s2 = integrate(integrate(rho_p * rho_p * sin(theta), (rho_p, 0, r)), (theta, 0, 2*sym_pi))
print("int_disk s2 dA =", I_disk_s2)

# 1D axis integral: int_0^1 x^a dt = (p0+p1)/2  (from rod first moment)
p_0a_r, p_1a_r = symbols('p_0^alpha p_1^alpha')
L = symbols('L', positive=True)  # segment length |e1|

# int_cylinder x^a dV = pi*r^2 * L * (p0+p1)/2 = V_cyl * midpoint
V_cyl = sym_pi * r**2 * L
midpoint = (p_0a_r + p_1a_r) / 2
first_moment_cyl = sym_pi * r**2 * L * midpoint
print("int_cylinder x^a dV =", sp.simplify(first_moment_cyl))
print("V_cyl * midpoint =", sp.simplify(V_cyl * midpoint))
print("Equal:", sp.simplify(first_moment_cyl - V_cyl * midpoint) == 0)


## Body Force Summary

For each primitive, with $f_{\text{density}} = \rho\, g_{\text{accel}}$ (N/m³):

### Shell (triangle $i$, slab thickness $2r$)

$$V_i = A_i \cdot 2r = \frac{|e_1 \times e_2|}{2} \cdot 2r = |e_1 \times e_2| \cdot r, \qquad c_i = \frac{p_0+p_1+p_2}{3}$$

$$\boxed{G[0:3]_i = f_{\text{density}} \cdot V_i}$$

$$\boxed{G[3:6]_i = f_x \cdot V_i \cdot c_i, \quad G[6:9]_i = f_y \cdot V_i \cdot c_i, \quad G[9:12]_i = f_z \cdot V_i \cdot c_i}$$

### Rod (segment $i$, cylinder radius $r$)

$$V_i = L_i \cdot \pi r^2, \qquad m_i = \frac{p_0+p_1}{2}$$

$$\boxed{G[0:3]_i = f_{\text{density}} \cdot V_i}$$

$$\boxed{G[3:6]_i = f_x \cdot V_i \cdot m_i, \quad G[6:9]_i = f_y \cdot V_i \cdot m_i, \quad G[9:12]_i = f_z \cdot V_i \cdot m_i}$$

Both formulas have the same structure as the existing 3D tet formula (`compute_tetmesh_body_force` in `compute_body_force.cpp`):
$$G[0:3] += f \cdot V, \quad G[3:12] += (f_x, f_y, f_z) \otimes (V \cdot c)$$

See implementation: `src/geometry/affine_body/compute_body_force.cpp` — `compute_codim_trimesh_body_force` and `compute_segment_body_force`.
